System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 8 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2023spring/slides/15-linreg`


Status `~/Documents/github.com/ucla-biostat-257/2023spring/slides/15-linreg/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [7522ee7d] SweepOperator v0.3.3
  [37e2e46d] LinearAlgebra


## Comparing methods for linear regression

Methods for solving linear regression $\widehat \beta = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$:

| Method            | Flops                  | Remarks                 | Software | Stability   |
| :---------------: | :--------------------: | :---------------------: | :------: | :---------: |
| Sweep             | $np^2 + p^3$           | $(X^TX)^{-1}$ available | SAS      | less stable |
| Cholesky          | $np^2 + p^3/3$         |                         |          | less stable |
| QR by Householder | $2np^2 - (2/3)p^3$     |                         | R        | stable      |
| QR by MGS         | $2np^2$                | $Q_1$ available         |          | stable      | 
| QR by SVD         | $4n^2p + 8np^2 + 9p^3$ | $X = UDV^T$             |          | most stable |  

Remarks:

1. When $n \gg p$, sweep and Cholesky are twice faster than QR and need less space.  
2. Sweep and Cholesky are based on the **Gram matrix** $\mathbf{X}^T \mathbf{X}$, which can be dynamically updated with incoming data. They can handle huge $n$, moderate $p$ data sets that cannot fit into memory.  
3. QR methods are more stable and produce numerically more accurate solution.  
4. Although sweep is slower than Cholesky, it yields standard errors and so on.  
5. MGS appears slower than Householder, but it yields $\mathbf{Q}_1$.

> **There is simply no such thing as a universal 'gold standard' when it comes to algorithms.**

## Benchmark

In [3]:
using SweepOperator, BenchmarkTools, LinearAlgebra

linreg_cholesky(y::Vector, X::Matrix) = cholesky!(X'X) \ (X'y)

linreg_qr(y::Vector, X::Matrix) = X \ y

function linreg_sweep(y::Vector, X::Matrix)
    p = size(X, 2)
    xy = [X y]
    tableau = xy'xy
    sweep!(tableau, 1:p)
    return tableau[1:p, end]
end

function linreg_svd(y::Vector, X::Matrix)
    xsvd = svd(X)
    return xsvd.V * ((xsvd.U'y) ./ xsvd.S)
end

linreg_svd (generic function with 1 method)

In [4]:
using Random

Random.seed!(123) # seed

n, p = 10, 3
X = randn(n, p)
y = randn(n)

# check these methods give same answer
@show linreg_cholesky(y, X)
@show linreg_qr(y, X)
@show linreg_sweep(y, X)
@show linreg_svd(y, X);

linreg_cholesky(y, X) = [-0.07196570434574735, -0.13575699455859386, -0.18820199689456507]
linreg_qr(y, X) = [-0.07196570434574738, -0.1357569945585939, -0.18820199689456502]
linreg_sweep(y, X) = [-0.07196570434574734, -0.1357569945585939, -0.188201996894565]
linreg_svd(y, X) = [-0.07196570434574735, -0.13575699455859377, -0.1882019968945651]


In [5]:
n, p = 1000, 300
X = randn(n, p)
y = randn(n)

@benchmark linreg_cholesky(y, X)

BenchmarkTools.Trial: 6158 samples with 1 evaluation.
 Range (min … max):  671.250 μs …  10.852 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     729.666 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   810.957 μs ± 556.360 μs  ┊ GC (mean ± σ):  1.85% ± 6.78%

  ▁▆█▆▃▃▃▂▂▁▂▁▁▂▁                                               ▁
  ███████████████▇▇▇▆▅▅▅▅▆▅▃▅▃▃▄▅▅▇▇▄▅▁▅▆▆▅▆▅▅▃▅▅▅▃▅▅▅▄▃▄▄▄▄▃▃▅ █
  671 μs        Histogram: log(frequency) by time       1.92 ms <

 Memory estimate: 708.17 KiB, allocs estimate: 4.

In [6]:
@benchmark linreg_sweep(y, X)

BenchmarkTools.Trial: 1047 samples with 1 evaluation.
 Range (min … max):  4.528 ms …   6.303 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.713 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.776 ms ± 206.955 μs  ┊ GC (mean ± σ):  0.96% ± 3.21%

       ▂▃▅██▆▅▆▄▄▃ ▁                                           
  ▆▅▅▆███████████████▆▅▄▄▄▅▁▄▄▁▄▁▄▁▁▁▁▁▁▁▅▇▆▆█▆▅▇▁▅▆▇▆▆▁▆▅▆▄▅ █
  4.53 ms      Histogram: log(frequency) by time      5.63 ms <

 Memory estimate: 2.99 MiB, allocs estimate: 6.

In [7]:
@benchmark linreg_qr(y, X)

BenchmarkTools.Trial: 681 samples with 1 evaluation.
 Range (min … max):  6.954 ms …  19.320 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.109 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.342 ms ± 967.909 μs  ┊ GC (mean ± σ):  1.27% ± 3.70%

  ▂▆▇█▇▅▄▂  ▁       ▁▁                                         
  ███████████▆█▆██▆█████▇▁▄▇▆▅▇▇▅▇▄█▅▅▄▅▅▁▇▇▅▅▅▄▄▁▁▄▄▄▁▁▁▁▁▁▄ █
  6.95 ms      Histogram: log(frequency) by time      9.09 ms <

 Memory estimate: 3.95 MiB, allocs estimate: 1828.

In [8]:
@benchmark linreg_svd(y, X)

BenchmarkTools.Trial: 216 samples with 1 evaluation.
 Range (min … max):  21.427 ms … 77.298 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     21.886 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   23.163 ms ±  4.762 ms  ┊ GC (mean ± σ):  0.80% ± 1.95%

  ▅█▂▃▂                                                        
  ██████▅▄▅▄▅▄▁▁▁▁▄▄▁▄▁▄▁▄▁▁▄▄▁▁▄▄▁▁▁▁▄▁▄▄▁▁▆▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▄ ▅
  21.4 ms      Histogram: log(frequency) by time      39.2 ms <

 Memory estimate: 8.06 MiB, allocs estimate: 14.